In [ ]:
!pip install requests beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from pprint import pprint

In [53]:
with open('resources/members.txt', 'r') as f:
    members = f.read().splitlines() 

In [3]:
URL_ROOT = 'https://forum.egosoft.com/'
URL_FORUM = 'viewforum.php?f='
URL_TOPIC = 'viewtopic.php?t='
FORUM_X4 = '146'

Parse main page

In [4]:
response = requests.get(URL_ROOT + URL_FORUM + FORUM_X4)

In [5]:
content = BeautifulSoup(response.content, 'html5lib')

In [6]:
topicsRaw = content.findAll(['a'], {'class': 'topictitle'})

In [16]:
import re

In [35]:
def topicInfo(topic):
    m = re.search('.*t=(\d*).*', topic['href'])    
    return {
        'id': m.group(1),
        'title': topic.getText(),
        'posts': topic.parent.parent.parent.find('dd',{'class','posts'}).getText().split(' ', 1)[0]
    }

In [36]:
topics = [topicInfo(t) for t in topicsRaw]

In [37]:
topics

[{'id': '400256',
  'posts': '89',
  'title': 'NEWS: Forum Update week of Sept 24th-28th, Downtimes'},
 {'id': '400042', 'posts': '309', 'title': 'Screenshot of the Day'},
 {'id': '396308', 'posts': '0', 'title': 'Welcome! - Please Read This First'},
 {'id': '396083', 'posts': '6', 'title': 'X4: Foundations - Info'},
 {'id': '396185',
  'posts': '1',
  'title': '[Submit] Questions about X4: Foundations (Now Locked)'},
 {'id': '396292',
  'posts': '6',
  'title': '[INDEX] X4: Foundations - Announcement Reaction Q&A'},
 {'id': '400381',
  'posts': '6',
  'title': 'How the station construction will work ?'},
 {'id': '400389',
  'posts': '1',
  'title': 'Anything been said of bailing or salvage?'},
 {'id': '400354',
  'posts': '16',
  'title': 'Temporary orders > ship resumes prior task once complete.'},
 {'id': '396144',
  'posts': '48',
  'title': 'X4 VR & Head Tracking control capability?'},
 {'id': '400128',
  'posts': '34',
  'title': 'Question about how large areas of space are handl

In [38]:
topic = topics[0]

In [39]:
topic

{'id': '400256',
 'posts': '89',
 'title': 'NEWS: Forum Update week of Sept 24th-28th, Downtimes'}

Parse topic pages

In [90]:
URL_ROOT + URL_TOPIC + topic['id'] + '&start=30'

'https://forum.egosoft.com/viewtopic.php?t=400256&start=30'

In [95]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=30')

In [96]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [97]:
blocksToConsider = []

for n in tContent.findAll('p', {'class': 'author'}):
    name = n.find('strong').getText().strip()
    if name in members:
        print (name)
        blocksToConsider.append(n)

BurnIt!
CBJ
CBJ
BurnIt!


In [99]:
blockToConsider = blocksToConsider[2]

Parse block 

In [100]:
post = blockToConsider.parent

In [101]:
post

<div id="post_content4731485">

						<h3><a href="#p4731485"></a></h3>

													<ul class="post-buttons">
																																									<li>
							<a class="button button-icon-only" href="./posting.php?mode=quote&amp;f=146&amp;p=4731485&amp;sid=4844f23e78822990efcc29220e4ff86f" title="Reply with quote">
								<i aria-hidden="true" class="icon fa-quote-left fa-fw"></i><span class="sr-only">Quote</span>
							</a>
						</li>
														</ul>
							
						<p class="author">
									<a class="unread" href="./viewtopic.php?p=4731485&amp;sid=4844f23e78822990efcc29220e4ff86f#p4731485" title="Post">
						<i aria-hidden="true" class="icon fa-file fa-fw icon-lightgray icon-md"></i><span class="sr-only">Post</span>
					</a>
								<span class="responsive-hide">by <strong><a class="username-coloured" href="./memberlist.php?mode=viewprofile&amp;u=39606&amp;sid=4844f23e78822990efcc29220e4ff86f" style="color: #FFCC00;">CBJ</a></strong> » </span>Sat, 22. Sep 18, 10:56
			

In [102]:
postId = post['id'][12:]

In [103]:
postId

'4731485'

In [104]:
publicationDate = re.findall('» (.*)', post.find('p', {'class': 'author'}).getText().strip())[0]
print(publicationDate)

Sat, 22. Sep 18, 10:56


In [105]:
postBody = post.find('div', {'class': 'content'})

In [106]:
postBody

<div class="content"><blockquote><div><cite>Turmfalke2 wrote:</cite>Right now both egosoft.com and the forums default to http. Since you are already using Let's Encrypt there is little reason not to redirect all http traffic to https. Currently most of the links within the forum are https, but some like the FAQ aren't. If you are using certbot with webroot-path you want to exclude .well-known/acme-challenge so the auto renew still works.</div></blockquote>
The forum has been set to default to https for quite some time. Are you sure you're not just using an old bookmark?<br/>

<blockquote><div><cite><a class="postlink" href="https://www.egosoft.com/games/x4/info_en.php">https://www.egosoft.com/games/x4/info_en.php</a> wrote:</cite>Encrypted payments on this website</div></blockquote>
This is most likely just an outdated message, but we can check. Thanks for highlighting it.</div>

In [111]:
for c in postBody.children:
    if str(c).strip() == '':
        continue
    print(c)
    print('---------------------')

<blockquote><div><cite>Turmfalke2 wrote:</cite>Right now both egosoft.com and the forums default to http. Since you are already using Let's Encrypt there is little reason not to redirect all http traffic to https. Currently most of the links within the forum are https, but some like the FAQ aren't. If you are using certbot with webroot-path you want to exclude .well-known/acme-challenge so the auto renew still works.</div></blockquote>
---------------------

The forum has been set to default to https for quite some time. Are you sure you're not just using an old bookmark?
---------------------
<br/>
---------------------
<blockquote><div><cite><a class="postlink" href="https://www.egosoft.com/games/x4/info_en.php">https://www.egosoft.com/games/x4/info_en.php</a> wrote:</cite>Encrypted payments on this website</div></blockquote>
---------------------

This is most likely just an outdated message, but we can check. Thanks for highlighting it.
---------------------


In [ ]:
for pb in postBody.children:
    if str(pb) == '':
        continue

    text = pb.getText().strip()
    
    chunk = {
        'type': pb.attrs['class'],
        'text': text,
        'images': [],
        'links': []
    }
    
    for img in pb.findAll('img'):
        chunk['images'].append(img['src'])
    
    for a in pb.findAll('a', {'class': 'postlink'}):
        chunk['links'].append(a)
    
    pprint(chunk)

If we reach the last page, the next one will produce a general error!

In [112]:
topic['id']

'400256'

In [117]:
URL_ROOT + URL_TOPIC + topic['id'] + '&start=100'

'https://forum.egosoft.com/viewtopic.php?t=400256&start=100'

In [113]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=100')

In [114]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [115]:
'General Error' in tContent.getText()

False

In [116]:
tContent

<!DOCTYPE html>
<html dir="ltr" lang="en-gb"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>

<link href="/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<title>NEWS: Forum Update week of Sept 24th-28th, Downtimes - Page 7 - egosoft.com</title>

	<link href="/app.php/feed?sid=9dee97e769c475d67d90ea9bab9b6636" rel="alternate" title="Feed - egosoft.com" type="application/atom+xml"/>			<link href="/app.php/feed/topics?sid=9dee97e769c475d67d90ea9bab9b6636" rel="alternate" title="Feed - New Topics" type="application/atom+xml"/>		<link href="/app.php/feed/forum/146?sid=9dee97e769c475d67d90ea9bab9b6636" rel="alternate" title="Feed - Forum - X4: Foundations" type="application/atom+xml"/>	<link href="/app.php/feed/topic/400256?sid=9dee97e769c475d67d90ea9bab9b6636" rel="alternate" title="Feed - Topic - NEWS: Forum Update week of Sept 24th-28th, Downtimes" type="applicat